In [2]:
# Lets Import all our packages nesscary for data importing
import datetime
import os
import shutil
from decimal import Decimal

import fsspec
import pandas as pd
from nautilus_trader.core.datetime import dt_to_unix_nanos
from nautilus_trader.model.data import QuoteTick
from nautilus_trader.model.objects import Price, Quantity

from nautilus_trader.backtest.node import BacktestNode, BacktestVenueConfig, BacktestDataConfig, BacktestRunConfig, BacktestEngineConfig
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.test_kit.providers import TestInstrumentProvider


In [6]:
from nautilus_trader.persistence.catalog import ParquetDataCatalog
catalog = ParquetDataCatalog.from_env()
catalog.instruments()

[CurrencyPair(id=EUR/USD.SIM, raw_symbol=EUR/USD, asset_class=FX, asset_type=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None)]

### Writing a AMA Trading Strategy

In [33]:
from typing import Optional
from nautilus_trader.core.message import Event
from nautilus_trader.indicators.average.hma import HullMovingAverage
from nautilus_trader.model.data import QuoteTick
from nautilus_trader.model.enums import PriceType
from nautilus_trader.model.enums import PositionSide
from nautilus_trader.model.enums import OrderSide
from nautilus_trader.model.events import PositionOpened
from nautilus_trader.model.identifiers import InstrumentId
from nautilus_trader.model.objects import Quantity
from nautilus_trader.model.position import Position
from nautilus_trader.trading.strategy import Strategy, StrategyConfig

In [37]:
class HMAConfig(StrategyConfig):
    instrument_id = str
    period: int = 20
    price_type: PriceType = PriceType.MID
    trade_size: int = 1_000_000
    entry_threshold: float = 0.00010

class HMAStrategy(Strategy):
    def __init__(self, config: HMAConfig):
        super().__init__(config=config)
        # Our "trading signal"
        self.hma = HullMovingAverage(period = config.period,
        price_type= config.price_type)
        self.entry_threshold = config.entry_threshold
        self.instrument_id = InstrumentId.from_str(config.instrument_id)
        self.entry_threshold = config.entry_threshold
        self.trade_size = config.trade_size

    # Convenience
        self.position: Optional[Position] = None
    # On start
    def on_start(self):
        # subscribe to data  
        self.subscribe_quote_ticks(instrument_id=self.instrument_id)

    # On end
    def on_stop(self):
        # Close all positions
        self.close_all_positions(self.instrument_id)
        # stop subscribing to data
        self.unsubscribe_quote_ticks(instrument_id=self.instrument_id)

    def on_quote_tick(self, tick:QuoteTick):
        # Updates indicator with quote tick 
        self.hma.handle_quote_tick(tick)
        if not self.hma.initialized:
            return # wait for indicator to start

        self.check_for_entry()
        self.check_for_exit()

    def on_event(self, event: Event):
        if isinstance(event, PositionOpened):
            self.position = self.cache.position(event.position_id)

    def check_for_entry(self):
        if self.macd.value > self.entry_threshold:
            if selfmposition and self.position.side == PositionSide.LONG:
                return # already LONG

            order = self.order_factory.market(
                instrument_id = self.instrument_id,
                order_side = OrderSide.BUY,
                quantity = self.trade_size,
            )
            self.submit_order(order)
    
    


